### This notebook is for explaination of the rationale behind certain model decisions and error analysis.

### 1. What is learnt from the EDA

1. Tiny ImageNet is perfectly balanced with 500 images per class for training and 50 per class for validation. Accuracy is a reliable metric.

2. The model is at risk to "overfitting" there is not a lot of images per class.

3. Classes are not random; they belong to broader semantic groups (e.g., several types of dogs, several types of birds, several types of vehicles).

### 2. Modeling approaches

#### 2.0 Data Augmentation

Strategy for Fighting Overfitting: Data Augmentation
Since our EDA showed that 500 images per class is a very small sample for a 200-class problem, the model is highly prone to "memorizing" the training set. To fix this, I designed three levels of data augmentation to see how much "noise" the model can handle while still learning useful features.

1. Baseline (The Control Group)
What it is: Only basic conversion to Tensor and Normalization. No flips, no crops, no rotations.

2. Standard (Geometric Invariance)
What it is: Added RandomHorizontalFlip and RandomRotation(15).


3. Heavy (Feature Robustness & Occlusion)
What it is: Added RandomResizedCrop, ColorJitter, and RandomErasing.

In [ ]:
from viz import alex_res_viz

alex_res_viz()

FileNotFoundError: [Errno 2] No such file or directory: 'outputs/results_baseline_ResNet18.csv'

#### 2.1 Experimented Architecture

1. **AlexNet inspired architecture** - A bit outdated but good as a baseline for what modern NNs should be able to achieve. Have to be modify a bit to deals with the images size.  

2. **ResNet inspired architecture** - Modern NNs which should be a good architecture for general image classification tasks.  

3. **Hierarchical ResNet architecture** - By exploiting the hierarchical nature of data, this architecture (a few linear layers on top of ResNet) should be able to learn both what constitute a given class e.g., Canine, Feline and Coarser group e,g,. mammalia at the same time, additionally the NNs should make the features space after Gradient descent of the same coarse group closer together to help with task 2 Image search.  
In short the architecture learn a distance-based feature space, in addition to build a decision boundry between classes, cluster sub-classes together. The optimizer there fore optimized the the following;  
**Cross-Entropy Loss (The Classifier)**: Drives the three heads ($L_{Coarse}, L_{Mid}, L_{Fine}$) to separate the classes.  
**Center Loss (The Aggregator)**: It minimizes the distance between each sample and its class center in the 512-D feature space.   
**Total Loss Equation**: $$L_{total} = \lambda_0 L_{center} + \lambda_1 L_{coarse} + \lambda_2 L_{mid} + \lambda_3 L_{fine}$$

Based on:
Wen et al., 2016 (https://kpzhang93.github.io/papers/eccv2016.pdf)

#### 2.2 The Metrics